<a href="https://colab.research.google.com/github/Danie261/Trading-Algoritmico/blob/main/Indicadores%20T%C3%A9cnicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INDICADORES TÉCNICOS

In [ ]:
!pip install yfinance
!pip install talib-binary
!pip install seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import talib as ta
import seaborn as sns
sns.set()

ticker="TSLA"
info=yf.Ticker(ticker).info
longname=info["longName"]
if "," in longname:
    longname=longname.replace(",","")

##Coding 'Williams Percentage Range' Indicator

In [ ]:
df=yf.download(ticker,"2020-6-1")


#Cálculo de 'Williams % Range'
t=21
max_14=df["High"].rolling(t).max()
min_14=df["Low"].rolling(t).min()
wr=(-(max_14-df["Close"])/(max_14-min_14))*100
df["W%R"]=wr

df.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")
plt.grid()

plt.subplot(212)
plt.title("Williams % Range Chart")
plt.plot(df["W%R"])
plt.axhline(-20, linestyle="--", alpha=0.8, color="black")
plt.axhline(-50, linestyle="--", alpha=0.5, color="black")
plt.axhline(-80, linestyle="--", alpha=0.8, color="black")
plt.fill_between(df.index, df["W%R"], -20, where=df["W%R"]>-20, color="coral")
plt.fill_between(df.index, df["W%R"], -80, where=df["W%R"]<-80, color="lightgreen")
plt.xlabel("Date")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()
da["W%R"]=df["W%R"]

da["W%R (TA-Lib)"]=ta.WILLR(df["High"], df["Low"], df["Close"],21)
da.dropna(inplace=True)


plt.figure(figsize=(10,5))
plt.title("W%R Comparison")
plt.plot(da["W%R"], label="W%R")
plt.plot(da["W%R (TA-Lib)"], label="W%R (TA-Lib)")
plt.xlabel("Date")
plt.legend(loc="upper left")
plt.axhline(-20, linestyle="--", alpha=0.8, color="black")
plt.axhline(-50, linestyle="--", alpha=0.5, color="black")
plt.axhline(-80, linestyle="--", alpha=0.8, color="black")
plt.xlabel("Date")

plt.show()
print(da.head(20))

##Coding 'AROON' Indicator

In [ ]:
df=yf.download(ticker,"2021-8-1")

df["Index"]=range(len(df))
n=10


#Cálculo 'AROON'
class aroon():
    def a_up(self):
        a=df["High"].rolling(n).max()
        dif=[]
        aup=[]
        for i in range(len(df)):
            index=0
            if i<n:
                dif+=[np.nan]
                aup+=[np.nan]
            else:
                for j in range(i,i-n,-1):
                    if df["High"][j]==a[i]:
                        index=df["Index"][j]
                dif+=[i-index]
                aup+=[int(((n-(i-index))/n)*100)]
        return aup

    def a_down(self):
        a=df["Low"].rolling(n).min()
        dif=[]
        adown=[]
        for i in range(len(df)):
            index=0
            if i<n:
                dif+=[np.nan]
                adown+=[np.nan]
            else:
                for j in range(i,i-n,-1):
                    if df["Low"][j]==a[i]:
                        index=df["Index"][j]
                dif+=[i-index]
                adown+=[int(((n-(i-index))/n)*100)]
        return adown, dif

x=aroon()       
df["AROON Up"]=x.a_up()        
df["AROON Down"], df["dif"]=x.a_down()


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    for i in range(len(df)):
        if i==0:
            b+=[np.nan]
            s+=[np.nan]
        elif df["AROON Up"][i-1]<df["AROON Down"][i-1] and df["AROON Up"][i]>df["AROON Down"][i]:
            b+=[df["Close"][i]]
            s+=[np.nan]
        elif df["AROON Up"][i-1]>df["AROON Down"][i-1] and df["AROON Up"][i]<df["AROON Down"][i]:
            b+=[np.nan]
            s+=[df["Close"][i]]
        else:
            b+=[np.nan]
            s+=[np.nan]
    return b, s

df["Buy"], df["Sell"]=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price", alpha=0.7)
plt.scatter(df.index, df["Buy"], label="Buy Price", marker="^", color="green")
plt.scatter(df.index, df["Sell"], label="Sell Price", marker="v", color="red")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")


plt.subplot(212)
plt.plot(df["AROON Up"], label="AROON Up")
plt.plot(df["AROON Down"], label="AROON Down")
plt.axhline(50, color="black", lw=0.5)
plt.axhline(70, color="black", lw=0.5)
plt.axhline(30, color="black", lw=0.5)
plt.title("AROON Chart")
plt.xlabel("Date")
plt.legend(loc="upper left")

plt.show()

Comparación con TA-Lib

In [ ]:
da=pd.DataFrame()

da["AROON Down"]=df["AROON Down"]
da["AROON Up"]=df["AROON Up"]

da["AROON Down (TA-Lib)"], da["AROON Up (TA-Lib)"]=ta.AROON(df["High"], df["Low"], n)
da.dropna(inplace=True)


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(da["AROON Up"], label="AROON Up")
plt.plot(da["AROON Down"], label="AROON Down")
plt.axhline(50, color="black", lw=0.5)
plt.title("AROON Chart")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(da["AROON Up (TA-Lib)"], label="AROON Up (TA-Lib)")
plt.plot(da["AROON Down (TA-Lib)"], label="AROON Down (TA-Lib)")
plt.axhline(50, color="black", lw=0.5)
plt.title("AROON (TA-Lib) Chart")
plt.legend(loc="upper left")
plt.xlabel("Date")

plt.show()
print(da.head(20))

##Coding 'ADX' Indicator

In [ ]:
df=yf.download(ticker,"2020-6-1")

#Cálculo 'ATR'
class calculo_ATR:
    t=14

    def tr(self):
        tr=[]
        for i in range(len(df)):
            if i!=0:
                a=df["High"][i]-df["Low"][i]
                b=abs(df["High"][i]-df["Close"][i-1])
                c=abs(df["Low"][i]-df["Close"][i-1])
                tr+=[max(a,b,c)]
            else:
                tr+=[np.nan]
        return tr

    def a_tr(self):
        atr=[]
        atr_0=0
        for i in range(len(df)):
            if len(atr)==0:
                for j in range(1,self.t+1):
                    atr_0+=t_r[j]
                atr+=[atr_0/self.t]
            elif len(atr)==1:
                atr+=[(atr[0]*(self.t-1)+t_r[i])/self.t]
            else:
                atr+=[(atr[i-1]*(self.t-1)+t_r[i])/self.t]
        return atr

x=calculo_ATR()
t_r=x.tr()
df["ATR"]=x.a_tr()


#Cálculo '+DM y -DM'
def dms():
    pos_dm=[]
    neg_dm=[]
    for i in range(len(df)):
        if i!=0:
            upmove=df["High"][i]-df["High"][i-1]
            downmove=df["Low"][i]-df["Low"][i-1]
            if upmove>downmove:
                if upmove>0:
                    pos_dm+=[upmove]
                    neg_dm+=[0]
                else:
                    pos_dm+=[0]
                    neg_dm+=[0]
            elif upmove<downmove:
                if downmove>0:
                    neg_dm+=[downmove]
                    pos_dm+=[0]
                else:
                    neg_dm+=[0]
                    pos_dm+=[0]
            else:
                pos_dm+=[0]
                neg_dm+=[0]
        else:
            pos_dm+=[np.nan]
            neg_dm+=[np.nan]
    return pos_dm, neg_dm

df["+DM"], df["-DM"]=dms()

del(df["Open"],df["Adj Close"], df["Volume"])


#Cálculo '+DI y -DI'
t=14
def dis():
    pos_di=[]
    neg_di=[]
    a=df["+DM"].ewm(span=t,adjust=False, min_periods=t).mean()
    b=df["-DM"].ewm(span=t,adjust=False, min_periods=t).mean()

    for i in range(len(df)):
        if i>t-1:
            pos_di+=[(a[i]/df["ATR"][i])*100]
            neg_di+=[abs((b[i]/df["ATR"][i])*100)]
        else:
            pos_di+=[np.nan]
            neg_di+=[np.nan]

    return pos_di, neg_di

df["+DI"], df["-DI"]=dis()


#Cálculo 'DX'
def dx():#dmi
    dx=[]
    for i in range(len(df)):
        if i>t-1:
            dx+=[abs((df["+DI"][i]-df["-DI"][i])/(df["+DI"][i]+df["-DI"][i]))*100]
        else:
            dx+=[np.nan]
    return dx

df["DX"]=dx()
df.dropna(inplace=True)


#Cálculo de 'ADX'
def adx():
    adx=[]
    for i in range(len(df)):
        if len(adx)==0:
            adx_0=0
            for j in range(1, t+1):
                adx_0+=df["DX"][j]
            adx+=[adx_0/t]
        
        elif len(adx)==1:
            adx+=[(adx[0]*(t-1)+df["DX"][i])/t]
        else:
            adx+=[(adx[i-1]*(t-1)+df["DX"][i])/t]
    return adx

df["ADX"]=adx()


#Algoritmo de c/v
def signals():
    b=[]
    s=[]
    condition=0
    for i in range(len(df)):
        if df["ADX"][i-1]<25 and df["ADX"][i]>25 and df["+DI"][i]>df["-DI"][i]:
            if condition!=1:
                b+=[df["Close"][i]]
                s+=[np.nan]
                condition=1
                
            else:
                b+=[np.nan]
                s+=[np.nan]
                
        elif df["ADX"][i-1]<25 and df["ADX"][i]>25 and df["-DI"][i]>df["+DI"][i]:
            if condition!=-1:
                b+=[np.nan]
                s+=[df["Close"][i]]
                condition=-1
                
            else:
                b+=[np.nan]
                s+=[np.nan]
                
        else:
            b+=[np.nan]
            s+=[np.nan]
            
    return b, s

b,s=signals()


#Plotting
plt.figure(figsize=(17,11))
plt.subplot(211)
plt.plot(df["Close"], label="Stock Price", alpha=0.8)
plt.scatter(df.index, b, label="Buy Price", marker="^", color="green")
plt.scatter(df.index, s, label="Sell Price", marker="v", color="red")
plt.title(f"{longname} Stock Price")
plt.legend(loc="upper left")

plt.subplot(212)
plt.plot(df["ADX"], color="darkblue", label="ADX", alpha=0.5)
plt.plot(df["+DI"], color="green", label="+DI")
plt.plot(df["-DI"], color="red", label="-DI")
plt.axhline(25, color="black", linestyle ="--")
plt.xlabel("Date")
plt.legend(loc="upper left")
plt.title("ADX Chart")

plt.show()